In [1]:
import pandas as pd

train = pd.read_csv('../../data/train.csv')
test = pd.read_csv('../../data/test.csv')

train.head()

,ID,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,y
0,TRAIN_00000,1.006187,-1.962566,1.247535,0.926500,-0.265766,-1.789301,0.470004,-0.139467,0.623996,0.320359,0.078612,83.424500
1,TRAIN_00001,1.024647,-2.472625,1.144386,0.846499,-0.287336,-1.756679,0.503860,-0.219545,0.697607,0.238306,0.081778,79.374109
2,TRAIN_00002,1.062444,-2.451003,1.186546,0.873599,-0.257828,-1.802735,0.498045,-0.194247,0.684134,0.259392,0.095003,82.181616
3,TRAIN_00003,1.089189,-2.458470,1.184531,0.810867,-0.276517,-1.787739,0.503359,-0.201923,0.686394,0.245736,0.091737,83.006586
4,TRAIN_00004,1.023323,-2.133468,1.242266,0.939837,-0.264515,-1.792044,0.470478,-0.142896,0.623778,0.314610,0.078987,83.051434


In [2]:
train_X = train.drop(columns=['ID','y'])
train_y = train['y']

train_data = pd.concat([train_X, train_y], axis=1)

In [3]:
# normalize
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

train_X_nor = minmax.fit_transform(train_X)

train_X_nor = pd.DataFrame(train_X_nor, columns=train_X.columns)

train_data_nor = pd.concat([train_X_nor, train_y], axis=1) # 인덱스 불일치로 인한 결합 오류 방지


In [ ]:
import sweetviz as sv

report = sv.analyze(train_X_nor)
report.show_html('report.html')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# create histograms for each variables
fig,axes = plt.subplots(4,3,figsize=(20,20))
axes = axes.ravel() #  Converts a two-dimensional array to a one-dimensional array

for i, col in enumerate(train):
    sns.histplot(train[col], ax= axes[i], kde=True)
    axes[i].set_title(f'Distribution of {col}')

plt.tight_layout()
plt.show()
  
# create box-plots for each variable
fig,axes = plt.subplots(4,3,figsize=(20,20))
axes = axes.ravel()

for i, col in enumerate(train):
    sns.boxplot(y=train[col], ax= axes[i])
    axes[i].set_title(f'Box-plot of {col}')
    
plt.tight_layout()
plt.show()

In [5]:
from pycaret.regression import setup, compare_models, pull, save_model, load_model, predict_model

regressor = setup(data = train_data_nor, target='y', session_id=42)
best_model = compare_models()
results = pull()

results

,Description,Value
0,Session id,42
1,Target,y
2,Target type,Regression
3,Original data shape,"(40118, 12)"
4,Transformed data shape,"(40118, 12)"
5,Transformed train set shape,"(28082, 12)"
6,Transformed test set shape,"(12036, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,1.2056,2.8778,1.6879,0.6054,0.0220,0.0146,0.0180
br,Bayesian Ridge,1.2057,2.8778,1.6879,0.6054,0.0220,0.0146,0.0250
lr,Linear Regression,1.2057,2.8779,1.6879,0.6053,0.0220,0.0146,0.4210
gbr,Gradient Boosting Regressor,1.2084,2.8859,1.6903,0.6042,0.0220,0.0146,2.6440
lightgbm,Light Gradient Boosting Machine,1.2115,2.8939,1.6928,0.6031,0.0220,0.0147,0.4670
huber,Huber Regressor,1.2082,2.9308,1.7037,0.5980,0.0221,0.0146,0.1690
et,Extra Trees Regressor,1.2378,2.9867,1.7200,0.5902,0.0223,0.0150,1.7040
rf,Random Forest Regressor,1.2355,2.9942,1.7221,0.5892,0.0224,0.0150,6.5190
knn,K Neighbors Regressor,1.3263,3.4002,1.8379,0.5322,0.0237,0.0161,0.0410
omp,Orthogonal Matching Pursuit,1.3741,3.6648,1.9081,0.4961,0.0243,0.0165,0.0250


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,1.2056,2.8778,1.6879,0.6054,0.0220,0.0146,0.018
br,Bayesian Ridge,1.2057,2.8778,1.6879,0.6054,0.0220,0.0146,0.025
lr,Linear Regression,1.2057,2.8779,1.6879,0.6053,0.0220,0.0146,0.421
gbr,Gradient Boosting Regressor,1.2084,2.8859,1.6903,0.6042,0.0220,0.0146,2.644
lightgbm,Light Gradient Boosting Machine,1.2115,2.8939,1.6928,0.6031,0.0220,0.0147,0.467
huber,Huber Regressor,1.2082,2.9308,1.7037,0.5980,0.0221,0.0146,0.169
et,Extra Trees Regressor,1.2378,2.9867,1.7200,0.5902,0.0223,0.0150,1.704
rf,Random Forest Regressor,1.2355,2.9942,1.7221,0.5892,0.0224,0.0150,6.519
knn,K Neighbors Regressor,1.3263,3.4002,1.8379,0.5322,0.0237,0.0161,0.041
omp,Orthogonal Matching Pursuit,1.3741,3.6648,1.9081,0.4961,0.0243,0.0165,0.025


In [6]:
save_model(best_model, 'best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['x_0', 'x_1', 'x_2', 'x_3', 'x_4',
                                              'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
                                              'x_10'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model', Ridge(random_state=42))]),
 'best_model.pkl')

In [10]:
test_X = test.drop(columns=['ID'])
best_model = load_model('best_model')
preds = predict_model(best_model, data = pd.DataFrame(minmax.fit_transform(test_X),columns=test_X.columns))

preds

Transformation Pipeline and Model Successfully Loaded


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,prediction_label
0,0.790086,0.280229,0.874827,0.697656,0.955132,0.082243,0.027599,0.433499,0.446714,0.431566,0.917452,83.692986
1,0.801440,0.024497,0.492295,0.369242,0.920037,0.160339,0.300883,0.154273,0.952307,0.113980,0.931578,82.085640
2,0.343243,0.753077,0.547437,0.930580,0.483241,0.674365,0.430666,0.849188,0.053469,0.796961,0.396383,90.182465
3,0.342545,0.849160,0.418901,0.773911,0.359401,0.788301,0.594343,0.917238,0.043102,0.848166,0.273070,91.311996
4,0.789560,0.087976,0.601507,0.514655,0.911537,0.125983,0.326101,0.191016,0.924250,0.111883,0.949103,82.189911
...,...,...,...,...,...,...,...,...,...,...,...,...
4981,0.635549,0.263917,0.913764,0.531002,0.972736,0.041646,0.065299,0.407770,0.494692,0.386773,0.933122,83.235725
4982,0.124667,0.860206,0.423876,0.960795,0.322436,0.794500,0.617516,0.903612,0.066597,0.847050,0.270916,91.427254
4983,0.859478,0.112179,0.802064,0.483255,0.987994,0.019689,0.142806,0.311153,0.692450,0.312817,0.979670,82.735855
4984,0.805804,0.108137,0.904568,0.628552,0.975431,0.027429,0.128406,0.351571,0.742265,0.269240,0.994287,82.996490


In [13]:
submission = pd.read_csv('../../data/sample_submission.csv')
submission['y'] = preds['prediction_label']
submission.to_csv('../normalized_AutoML/nor_submission.csv',index=False)